# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into Deep Neural Networks for NLP!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of Word2Vec such as vector size and window size

## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec Model. Then, you'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what your data looks like. 

In [4]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data--`headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [5]:
df['combined_text'] = df.headline + ' ' +  df.short_description
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [6]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [7]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count`. 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [8]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55565722, 67352790)

Great! you now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which is stored inside of `model.wv`. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [9]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [10]:
wv.most_similar('Texas')

[('Ohio', 0.817750096321106),
 ('Louisiana', 0.8018538951873779),
 ('Florida', 0.7989729046821594),
 ('Michigan', 0.7879190444946289),
 ('Wisconsin', 0.7874444127082825),
 ('Georgia', 0.7844291925430298),
 ('Oregon', 0.7808618545532227),
 ('Pennsylvania', 0.7749639749526978),
 ('Oklahoma', 0.7721328735351562),
 ('Arkansas', 0.7720518708229065)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [11]:
wv.most_similar(negative='Texas')

[('Parent/Grandparent', 0.45167285203933716),
 ('once-reliable', 0.4065055847167969),
 ('Candidate\u200b', 0.40368887782096863),
 ('Warring', 0.3809281885623932),
 ('Rank-and-file', 0.3807392716407776),
 ('Admon', 0.380428671836853),
 ('Hunger-Free', 0.3708605170249939),
 ('Ex-Lions', 0.36795979738235474),
 ('Animal-rescue', 0.36519157886505127),
 ('Gang-Raping', 0.36448848247528076)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [12]:
wv['Texas']

array([-0.77919966,  0.8556044 ,  0.69426346, -0.4203063 , -0.12787285,
        0.23398808,  0.7051716 , -0.7360195 ,  3.2256734 ,  1.8025783 ,
       -1.6841755 ,  1.7884161 , -0.46371353, -1.1261257 ,  0.12372027,
        0.6555217 ,  1.0510392 ,  3.4760826 , -0.06919644,  0.72798806,
        1.2545395 , -0.41219118,  1.286895  , -0.32591963, -3.5475492 ,
       -1.407774  ,  0.9124315 ,  0.6299803 ,  2.5131202 ,  1.5756993 ,
       -1.6400541 , -0.8281949 , -1.9600769 , -0.95801526,  1.9227567 ,
       -1.4607128 , -0.13629192,  0.3712209 , -1.422416  , -0.07955043,
        0.5292857 , -1.2986401 , -3.402235  ,  0.1393978 ,  0.5391121 ,
       -1.197414  , -1.8340594 , -0.26249537,  0.73086977,  0.01146855,
       -0.06353327, -1.0670716 , -2.3594313 ,  1.6843258 , -2.0000658 ,
        0.99886835,  1.4263557 ,  0.7966874 ,  0.26649433,  1.1269299 ,
       -0.5211859 , -0.134774  , -0.6241054 ,  0.99057263,  2.3330777 ,
       -2.6285424 , -1.3490266 ,  1.7953757 , -0.8320546 ,  1.60

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [13]:
wv.vectors

array([[-1.24552272e-01,  5.68742193e-02, -6.55031264e-01, ...,
        -9.65874970e-01, -4.78670150e-01,  1.02485514e+00],
       [-3.51102278e-02,  3.08651999e-02, -1.35106134e+00, ...,
        -6.15288734e-01, -2.41603637e+00,  1.16025937e+00],
       [ 2.46691895e+00,  3.22861600e+00, -5.13508022e-01, ...,
         2.70507067e-01, -2.65692282e+00,  9.80182350e-01],
       ...,
       [ 1.10183265e-02, -7.10755363e-02,  1.01898694e-02, ...,
         2.61518583e-02,  6.98683038e-02,  1.69141650e-01],
       [ 6.44094050e-02, -4.94227633e-02, -9.95935127e-02, ...,
         1.88639686e-02,  7.22526237e-02,  2.54244693e-02],
       [-2.65434477e-02, -9.12567750e-02, -8.03569984e-03, ...,
        -1.27451844e-03,  4.24319617e-02,  1.22410208e-01]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `most_similar` function and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [14]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('queen', 0.6236600875854492),
 ('princess', 0.5977774858474731),
 ('crown', 0.5763854384422302),
 ('title', 0.5745823383331299),
 ('Americana', 0.5714659690856934),
 ('brunette', 0.571101725101471),
 ('color', 0.5648080110549927),
 ('reminiscent', 0.564174234867096),
 ('monster', 0.5612499117851257),
 ('yellow', 0.5598372220993042)]

As you can see from the output above, your model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As you can see from the word in first place, 'reminiscent', your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to created vectorized word embeddings!